In [1]:
import pyarrow as pa    
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
dataset_dir = os.path.abspath("datasets/clean")

dataset_paths = os.listdir(dataset_dir)
dataset_paths = [os.path.join(dataset_dir, dir) for dir in dataset_paths]
dataset_names = [os.path.basename(path).split('.')[0] for path in dataset_paths]

In [3]:
# read datasets from path
datasets = dict(zip(dataset_names,[pq.read_table(path) for path in dataset_paths]))

In [4]:
name = list(datasets.keys())
n_samples = [0] + [ datasets[key].__len__() for key in datasets ]
index = np.cumsum(n_samples)

datasets_source = {
    'source':[],
    'index_min':[],
    'index_max':[]
}

for i in range(len(name)):
    print(f'{name[i]} -> {index[i]} - {index[i+1]-1}')
    datasets_source['source'].append(name[i])
    datasets_source['index_min'].append(index[i])
    datasets_source['index_max'].append(index[i+1]-1)
    
datasets_source = pd.DataFrame(datasets_source)
datasets_source.to_csv("datasets/datasets_source.csv",)

gigaword -> 0 - 3995558
thaisum -> 3995559 - 4376426
tldr-17 -> 4376427 - 8224756
wikihow -> 8224757 - 8811186
xlsum -> 8811187 - 9149046


In [5]:
# concat datasets into one table
dataset = pa.concat_tables(list(datasets.values()))
dataset.__len__()

9149047

In [6]:
# add column index.
ind = list(np.arange(dataset.__len__(), dtype=int))
dataset = dataset.add_column(0,'id', [ind])
dataset

pyarrow.Table
id: int32
text: string
sum: string
----
id: [[0,1,2,3,4,...,9149042,9149043,9149044,9149045,9149046]]
text: [["australia  is current account deficit shrunk by a record #.## billion dollars #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .","at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said .","australian shares closed down #.# percent monday following a weak lead from the united states and lower commodity prices , dealers said .","south korea  is nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its "" typical "" brinkmanship in negotiations .","south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment .",...,"south korea  is posco , the world  is

### randomly split the datasets into equally subsets

In [7]:
n_slice = 10
mask = np.random.randint(0, n_slice, dataset.__len__())

# create empty slices of datasets
slices = dict([ (f'slice_{i}',[]) for i in range(n_slice)])

for i_s in tqdm(range(n_slice), desc="geterate random split"):
    slices[f'slice_{i_s}'] = dataset.filter(mask==i_s)

geterate random split: 100%|██████████| 10/10 [06:37<00:00, 39.71s/it]


### write table datasets

In [13]:
for i in tqdm(range(n_slice), desc="save splited data"):
    pq.write_table(slices[f'slice_{i}'], where=os.path.abspath(f"datasets/slice_{i}.parquet"))

save splited data: 100%|██████████| 10/10 [02:07<00:00, 12.78s/it]


In [1]:
pa.dataset.Dataset(slices[0])

NameError: name 'pa' is not defined